# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [2]:
# Hacé la separación en esta celda

X =  df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [3]:
# Esto es lo que hace que se obtenga siempre la misma separacion aleatoria.
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [4]:
# Creá en esta celda la variable param_grid

param_grid = {'max_depth':np.arange(1, 6),
              'max_features':np.arange(1, 6)}

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [5]:
# Importa y crea un GridSearchCV en esta celda

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

rgr_tree = DecisionTreeRegressor()
grid_search = GridSearchCV(rgr_tree, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)



A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [6]:
# Hace el fit de grid search en esta celda

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': array([1, 2, 3, 4, 5]),
                         'max_features': array([1, 2, 3, 4, 5])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
        

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [7]:
grid_search.scorer_

make_scorer(mean_squared_error, greater_is_better=False)

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [8]:
# Mostrá los grid_scores en esta celda

scores = pd.DataFrame(grid_search.cv_results_)
scores

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008,0.002,0.002,0.001,1,1,"{'max_depth': 1, 'max_features': 1}",-997441086.490,-978211341.636,-934144133.967,-1005179382.551,-924101444.456,-967815477.820,32943855.755,21
1,0.005,0.000,0.001,0.000,1,2,"{'max_depth': 1, 'max_features': 2}",-992552006.629,-987399014.877,-932898809.311,-1002601147.219,-924101444.456,-967910484.499,32666493.225,23
2,0.006,0.001,0.001,0.000,1,3,"{'max_depth': 1, 'max_features': 3}",-994132879.425,-993157870.955,-917436842.919,-996809161.879,-965625386.208,-973432428.277,30201089.139,24
3,0.007,0.002,0.001,0.000,1,4,"{'max_depth': 1, 'max_features': 4}",-999271968.613,-737635506.111,-919493503.581,-1002601147.219,-971492632.752,-926098951.655,98822633.916,16
4,0.005,0.000,0.001,0.000,1,5,"{'max_depth': 1, 'max_features': 5}",-995065161.583,-986070141.632,-935566509.292,-1005934580.414,-968519006.170,-978231079.818,24604516.240,25
5,0.005,0.000,0.001,0.000,2,1,"{'max_depth': 2, 'max_features': 1}",-996667113.402,-995845633.089,-933708300.370,-797099480.210,-972642270.593,-939192559.533,74635512.386,18
6,0.005,0.000,0.002,0.000,2,2,"{'max_depth': 2, 'max_features': 2}",-698604357.118,-971213141.733,-929726211.324,-1004666582.349,-971530520.212,-915148162.547,110850835.637,14
7,0.005,0.000,0.001,0.000,2,3,"{'max_depth': 2, 'max_features': 3}",-702405254.753,-991561108.947,-915729396.108,-1005812094.471,-949579301.387,-913017431.133,109983246.741,12
8,0.005,0.000,0.001,0.000,2,4,"{'max_depth': 2, 'max_features': 4}",-996188501.356,-731836823.514,-893072954.514,-997687543.721,-955537710.241,-914864706.669,99097488.983,13
9,0.005,0.000,0.002,0.000,2,5,"{'max_depth': 2, 'max_features': 5}",-985713746.868,-983003075.180,-831444347.483,-915097523.038,-919573293.119,-926966397.138,56411613.194,17


De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [9]:
# Mostrás los resultados en esta celda

print("Mejores parametros: "+str(grid_search.best_params_))
print("Mejor Score: "+str(grid_search.best_score_)+'\n')


Mejores parametros: {'max_depth': 4, 'max_features': 4}
Mejor Score: -713698124.9712428



Convertimos a RMSE.

In [10]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [11]:
# Defino un nuevo param_grid:
param_grid = {"min_samples_split": [2, 10, 20],
              "max_depth": [None, 2, 5, 10, 15],
              "min_samples_leaf": [1, 5, 10, 15],
              "max_leaf_nodes": [None, 5, 10, 20]}

# Defino un nuevo modelo y grid_search:

rgr_tree = DecisionTreeRegressor()
grid_search = GridSearchCV(rgr_tree, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Entreno el nuevo modelo:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [None, 2, 5, 10, 15],
                         'max_leaf_nodes': [None, 5, 10, 20],
                         'min_samples_leaf': [1, 5, 10, 15],
                         'min

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [12]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [13]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [14]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [15]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [16]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
